# Name Entity Recognition using Deep Learning 

* Upload the lab_resources and NERC_nn files to you Drive Account:
  * Lab_resource: https://www.cs.upc.edu/~turmo/mud/MUD.html#:~:text=Task%20data%20%26%20other%20stuff
  * NERC_nn code: https://www.cs.upc.edu/~turmo/mud/MUD.html#:~:text=Guidelines%20and%20code%20for%20task%204%20(NN%2Dbased%20NER)
* Before running the code, ensure that your Google Colab is set to use GPU:
  * Edit → Notebook Settings
* Mount your Drive disk unit:
  * Left-side menu → Files → Mount drive (the icon that looks like a folder with the Drive logo).


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Define the paths to the data and utils in your Drive unit:

In [1]:
# Directorios Albert

utilsdir='drive/MyDrive/clases/MUD/DEL4+5-7.06/NERC-nn/05-NERC-nn'
evaluatordir='drive/MyDrive/clases/MUD/DEL4+5-7.06/NERC-nn/lab_resources/DDI/util'
traindir='drive/MyDrive/clases/MUD/DEL4+5-7.06/NERC-nn/lab_resources/DDI/data/train'
validationdir='drive/MyDrive/clases/MUD/DEL4+5-7.06/NERC-nn/lab_resources/DDI/data/devel'
testdir='drive/MyDrive/clases/MUD/DEL4+5-7.06/NERC-nn/lab_resources/DDI/data/test'
modelname ='model'
outfile ='drive/MyDrive/clases/MUD/DEL4+5-7.06/NERC-nn/05-NERC-nn/results/out.txt'

In [ ]:
# Directorios Maria
utilsdir='drive/MyDrive/MUD/DEL4+5-7.06/NERC-nn/05-NERC-nn'
evaluatordir='drive/MyDrive/MUD/DEL4+5-7.06/NERC-nn/lab_resources/DDI/util'
traindir='drive/MyDrive/MUD/DEL4+5-7.06/NERC-nn/lab_resources/DDI/data/train'
validationdir='drive/MyDrive/MUD/DEL4+5-7.06/NERC-nn/lab_resources/DDI/data/devel'
testdir='drive/MyDrive/MUD/DEL4+5-7.06/NERC-nn/lab_resources/DDI/data/test'
modelname ='model'
outfile ='drive/MyDrive/MUD/DEL4+5-7.06/NERC-nn/05-NERC-nn/results/out.txt'

In [ ]:
# Directorios Ximena
utilsdir='drive/MyDrive/MUD/lab5/05-NERC-nn'
evaluatordir='drive/MyDrive/MUD/lab5/lab_resources/DDI/util'
traindir='drive/MyDrive/MUD/lab5/lab_resources/DDI/data/train'
validationdir='drive/MyDrive/MUD/lab5/lab_resources/DDI/data/devel'
testdir='drive/MyDrive/MUD/lab5/lab_resources/DDI/data/test'
modelname ='model'
outfile ='drive/MyDrive/MUD/lab5/05-NERC-nn/results/out.txt'

In [2]:
!pip install tensorflow-addons
import sys
sys.path.insert(1,utilsdir) # Path to the utils folder on your Google Drive disk
sys.path.insert(1,evaluatordir) # Path to the evaluator folder on your Google Drive disk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from contextlib import redirect_stdout
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, GRU, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, concatenate, Softmax, ReLU
from tensorflow_addons.text.crf_wrapper import CRFModelWrapper

import tensorflow as tf
import nltk
nltk.download('punkt')
from codemaps import *


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:

def build_network(codes, max_len=50, suf_len=4) :

   # sizes
   n_words = codes.get_n_words()
   n_sufs = codes.get_n_sufs()
   n_lens = codes.get_n_lens()
   n_pres = codes.get_n_pres()
   n_dshs = codes.get_n_dshs()
   n_nums = codes.get_n_nums()
   n_loks = codes.get_n_loks()
   n_labels = codes.get_n_labels()
   max_len = codes.maxlen

   drop=0.2

   ######################################
   inptW = Input(shape=(max_len,))
   inptS = Input(shape=(max_len,))
   inptL = Input(shape=(max_len,))
   inptP = Input(shape=(max_len,))
   inptD = Input(shape=(max_len,))
   inptN = Input(shape=(max_len,))
   inptK = Input(shape=(max_len,))
   model1 = Embedding(input_dim=n_words, output_dim=1500,
                      input_length=max_len, mask_zero=False)(inptW)  # word embeddings
   model2 = Embedding(input_dim=n_sufs, output_dim=1000,
                      input_length=max_len, mask_zero=False)(inptS)  # suf embeddings
   model3 = Embedding(input_dim=n_lens, output_dim=300,
                      input_length=max_len, mask_zero=False)(inptL)  # len embeddings
   model4 = Embedding(input_dim=n_pres, output_dim=1000,
                      input_length=max_len, mask_zero=False)(inptP)  # pre embeddings
   model5 = Embedding(input_dim=n_dshs, output_dim=300,
                      input_length=max_len, mask_zero=False)(inptD)  # len embeddings
   model6 = Embedding(input_dim=n_nums, output_dim=300,
                      input_length=max_len, mask_zero=False)(inptN)  # pre embeddings
   model7 = Embedding(input_dim=n_loks, output_dim=300,
                      input_length=max_len, mask_zero=False)(inptK)  # pre embeddings
   model1 = Dropout(drop)(model1)
   model2 = Dropout(drop)(model2)
   model3 = Dropout(drop)(model3)
   model4 = Dropout(drop)(model4)
   model5 = Dropout(drop)(model5)
   model6 = Dropout(drop)(model6)
   model7 = Dropout(drop)(model7)
   model = concatenate([model1,model2,model3,model4,model5,model6,model7])
   y = Bidirectional(LSTM(units=50, return_sequences=True,dropout=0.2))(model)  #  biLSTM
   y = TimeDistributed(Dense(2*n_labels, activation=ReLU()))(y) 
   y = TimeDistributed(Dense(n_labels, activation=Softmax()))(y) 

   return Model(
        inputs=[inptW,inptS,inptL,inptP,inptD,inptN,inptK], outputs=y
    )
   

In [21]:
# directory with files to process
from dataset import *

# load train and validation data
traindata = Dataset(traindir)
valdata = Dataset(validationdir)

# create indexes from training data
max_len = 50
suf_len = 3
codes  = Codemaps(traindata, max_len, suf_len)

# encode datasets
[Xt,Xts,Xtl,Xtp,Xtd,Xtn,Xtk] = codes.encode_words(traindata)
Yt = codes.encode_labels(traindata)
[Xv,Xvs,Xvl,Xvp,Xvd,Xvn,Xvk] = codes.encode_words(valdata)
Yv = codes.encode_labels(valdata)

n_tags = codes.get_n_labels()
max_len = codes.maxlen

In [22]:
import tensorflow as tf

# set seed
tf.random.set_seed(3103)

model = build_network(codes)
model.compile(optimizer='adam' ,metrics=["accuracy"], loss="sparse_categorical_crossentropy")
model.build([(None,max_len),(None,max_len),(None,max_len),(None,max_len),(None,max_len),(None,max_len),(None,max_len)])

with redirect_stdout(sys.stderr) :
   model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_22 (InputLayer)          [(None, 50)]         0           []                               
                                                                                                  
 input_23 (InputLayer)          [(None, 50)]         0           []                               
                                                                                                  
 input_24 (InputLayer)          [(None, 50)]         0           []                               
                                                                                                  
 input_25 (InputLayer)          [(None, 50)]         0           []                               
                                                                                            

In [ ]:
## --------- MAIN PROGRAM ----------- 
## --
## -- Usage:  train.py ../data/Train ../data/Devel  modelname
## --


# train model
with redirect_stdout(sys.stderr) :
   model.fit([Xt,Xts,Xtl,Xtp,Xtd,Xtn,Xtk], Yt, batch_size=64, epochs=9, validation_data=([Xv,Xvs,Xvl,Xvp,Xvd,Xvn,Xvk],Yv), verbose=1)

# save model and indexes
model.save(modelname)
codes.save(modelname)
#save_model_and_indexes(model, idx, modelname)

Epoch 1/9
85/85 [==============================] - 39s 383ms/step - loss: 0.2759 - accuracy: 0.9479 - val_loss: 0.0888 - val_accuracy: 0.9740
Epoch 2/9
85/85 [==============================] - 16s 184ms/step - loss: 0.0607 - accuracy: 0.9821 - val_loss: 0.0549 - val_accuracy: 0.9859
Epoch 3/9
85/85 [==============================] - 10s 119ms/step - loss: 0.0291 - accuracy: 0.9925 - val_loss: 0.0463 - val_accuracy: 0.9884
Epoch 4/9
85/85 [==============================] - 10s 119ms/step - loss: 0.0172 - accuracy: 0.9957 - val_loss: 0.0436 - val_accuracy: 0.9893
Epoch 5/9
85/85 [==============================] - 6s 72ms/step - loss: 0.0109 - accuracy: 0.9974 - val_loss: 0.0478 - val_accuracy: 0.9892
Epoch 6/9
85/85 [==============================] - 7s 87ms/step - loss: 0.0076 - accuracy: 0.9982 - val_loss: 0.0482 - val_accuracy: 0.9895
Epoch 7/9
76/85 [=========================>....] - ETA: 0s - loss: 0.0056 - accuracy: 0.9986

# Predict 

In [8]:
#import sys
import evaluator

In [9]:
def output_entities(data, preds, outfile) :

   outf = open(outfile, 'w')
   for sid,tags in zip(data.sentence_ids(),preds) :
      inside = False
      for k in range(0,min(len(data.get_sentence(sid)),codes.maxlen)) :
         y = tags[k]
         token = data.get_sentence(sid)[k]

         if (y[0]=="B") :
             entity_form = token['form']
             entity_start = token['start']
             entity_end = token['end']
             entity_type = y[2:]
             inside = True
         elif (y[0]=="I" and inside) :
             entity_form += " "+token['form']
             entity_end = token['end']
         elif (y[0]=="O" and inside) :
             print(sid, str(entity_start)+"-"+str(entity_end), entity_form, entity_type, sep="|", file=outf)
             inside = False

      if inside : print(sid, str(entity_start)+"-"+str(entity_end), entity_form, entity_type, sep="|", file=outf)

   outf.close()

In [10]:
## --------- Evaluator ----------- 
def evaluation(datadir,outfile) :
   evaluator.evaluate("NER", datadir, outfile)


In [19]:
## --------- MAIN PROGRAM ----------- 
## --
## -- Usage:  baseline-NER.py target-dir
## --
## -- Extracts Drug NE from all XML files in target-dir
## --

datadir = validationdir

testdata = Dataset(datadir)
[X,Xs,Xl,Xp,Xd,Xn,Xk] = codes.encode_words(testdata)
Y = model.predict([X,Xs,Xl,Xp,Xd,Xn,Xk])
Y = [[codes.idx2label(np.argmax(w)) for w in s] for s in Y]

# extract entities
output_entities(testdata, Y, outfile)

# evaluate
evaluation(datadir,outfile)


45/45 [==============================] - 3s 41ms/step
                   tp	  fp	  fn	#pred	#exp	P	R	F1
------------------------------------------------------------------------------
brand             179	  23	 195	 202	 374	88.6%	47.9%	62.2%
drug             1704	 144	 202	1848	1906	92.2%	89.4%	90.8%
drug_n              6	   9	  39	  15	  45	40.0%	13.3%	20.0%
group             552	 108	 135	 660	 687	83.6%	80.3%	82.0%
------------------------------------------------------------------------------
M.avg            -	-	-	-	-	76.1%	57.7%	63.7%
------------------------------------------------------------------------------
m.avg            2441	 284	 571	2725	3012	89.6%	81.0%	85.1%
m.avg(no class)  2519	 206	 493	2725	3012	92.4%	83.6%	87.8%
